In [5]:
# Built-in Tools
# duckduckgo search
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

# this query will run on duckduckgo search engine and the results will be returned
results = search_tool.invoke("ipl news") 

print(results)

Get the Latest Cricket Updates, Check IPL 2025 News, Schedule, Points Table and Results at NDTV Sports. Like Us On Facebook Or Follow Us On Twitter For More Sports Updates. IPL 2025 Suspension Highlights: The Indian Premier League (IPL) was suspended indefinitely on Friday because of the escalating military confrontation between India and Pakistan. A cloud of uncertainty had loomed over the future of the ongoing edition since the cancellation of Thursday's match between Punjab Kings and Delhi Capitals in Dharamsala, midway following air raid alerts in ... Stay informed with all important IPL 2025 announcements, including player updates and squad changes. Get the latest news on IPLT20. There are 13 matches left to play in the IPL 2025 league stage, and a handful of teams are still in contention to make a late bid for a top-four finish. The teams have been rocked by a few departures, with players pulling out due to personal reasons or international schedule clashes. Get the latest news, 

In [7]:
# shell tool -> useful but risky (as files can also be deleted)
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

results = shell_tool.invoke("ls") # can run shell commands

print(results)

Executing command:
 ls
01_tools_in_langchain.ipynb



/Users/navdeep/Desktop/Generative AI/Generative-AI/langchain_indepth/01_langchain_models/venv/lib/python3.12/site-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


In [10]:
# Custom Tools
# Method 1 - using @tool decorator
from langchain_core.tools import tool

In [11]:
# Step - 1 create a function

def multiply(a, b):
    # highly recommended to add a docstring as it will help
    # the llm understand what this function does
    """Multiply two numbers""" 
    return a*b 

In [12]:
# Step 2 - add type hints
# recommended for the llm to understand what type of data
# is accepted by this function and what it can accept in return
def multiply(a: int, b: int) -> int:
    """Multiply two numbers""" 
    return a*b 

In [13]:
# Step 3 - add tool decorator

@tool 
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a*b

# now the llm can interact with above function as it is a tool now

In [14]:
result = multiply.invoke({"a": 2, "b": 3})

In [15]:
result

6

In [16]:
# let's see how the above function is a special function
print(multiply.name) # tool name is same as the function name
print(multiply.description) # prints the docstring
print(multiply.args) # description of arguments in the form of dictionary

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [17]:
# these attributes are available with builtin tools also
print(search_tool.name) 
print(search_tool.description) 
print(search_tool.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


In [18]:
# what llm sees when we send a tool to it
print(multiply.args_schema.model_json_schema())

{'description': 'Multiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


In [19]:
# the logic of the tool is not sent to the llm instead its schema
# is set to the llm in json format as shown below
{
   "description":"Multiply two numbers",
   "properties":{
      "a":{
         "title":"A",
         "type":"integer"
      },
      "b":{
         "title":"B",
         "type":"integer"
      }
   },
   "required":[
      "a",
      "b"
   ],
   "title":"multiply",
   "type":"object"
}

{'description': 'Multiply two numbers',
 'properties': {'a': {'title': 'A', 'type': 'integer'},
  'b': {'title': 'B', 'type': 'integer'}},
 'required': ['a', 'b'],
 'title': 'multiply',
 'type': 'object'}

In [20]:
# Method 2 - Using StructuredTool

from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [21]:
class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add.")
    b: int = Field(required=True, description="The second number to add.")

In [22]:
def multiply_func(a: int, b: int) -> int:
    return a * b

In [23]:
multiply_tool = StructuredTool.from_function(
    func=multiply_func,
    name="multiply",
    description="Multiply two numbers",
    args_schema=MultiplyInput
)

In [24]:
result = multiply_tool.invoke({"a": 3, "b": 2})

In [25]:
print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

6
multiply
Multiply two numbers
{'a': {'description': 'The first number to add.', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add.', 'required': True, 'title': 'B', 'type': 'integer'}}


In [26]:
# Method 3 - Using BaseTool class
from langchain.tools import BaseTool
from typing import Type

In [27]:
# arg schema using pydantic
class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add.")
    b: int = Field(required=True, description="The second number to add.")

In [28]:
class MultiplyTool(BaseTool): # inherits BaseTool class
    name: str = "mutiply"
    description: str = "Multiply two numbers"

    args_schema: Type[BaseModel] = MultiplyInput # defining the schema for arguments

    def _run(self, a: int, b: int) -> int:
        return a * b

In [29]:
multiply_tool = MultiplyTool()

In [30]:
result = multiply_tool.invoke({"a":3, "b":3})

In [31]:
print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

9
mutiply
Multiply two numbers
{'a': {'description': 'The first number to add.', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add.', 'required': True, 'title': 'B', 'type': 'integer'}}


Toolkit

In [32]:
from langchain_core.tools import tool

# Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    "Multiply two numbers"
    return a * b

In [33]:
class MathToolkit:

    def get_tools(self):
        return [add, multiply]

In [36]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

print("Tools")
for tool in tools:
    print(tool.name, "=>", tool.description)

Tools
add => Add two numbers
multiply => Multiply two numbers
